In [1]:
%load_ext lab_black

In [2]:
import requests
import json
import datetime
import pandas as pd
from pathlib import Path

In [3]:
# the UN's API includes many entities which are not sovereign states,
# yet are listed as "Country". This workaround is a manuel fix for if
# one is only interested in countries by the classic definition
not_countries = [
    "American Samoa",
    "Bermuda",
    "British Virgin Islands",
    "Cayman Islands",
    "Mayotte",
    "Cook Islands",
    "Faroe Islands",
    "Falkland Islands (Malvinas)",
    "French Guiana",
    "French Polynesia",
    "Gibraltar",
    "Greenland",
    "Guadeloupe",
    "Guam",
    "China, Hong Kong SAR",
    "China, Macao SAR",
    "Martinique",
    "Montserrat",
    "Curaçao",
    "Aruba",
    "Sint Maarten (Dutch part)",
    "Bonaire, Sint Eustatius and Saba",
    "New Caledonia",
    "Niue",
    "Northern Mariana Islands",
    "Puerto Rico",
    "Réunion",
    "Saint Helena",
    "Anguilla",
    "Saint Pierre and Miquelon",
    "Tokelau",
    "Turks and Caicos Islands",
    "Isle of Man",
    "United States Virgin Islands",
    "Wallis and Futuna Islands",
]

In [4]:
base_url = "https://population.un.org/dataportalapi/api/v1/indicators/"
response = requests.get(base_url)
print(response)

<Response [200]>


In [8]:
def callAPI(relative_path: str, topic_list: bool = False) -> pd.DataFrame:
    base_url = "https://population.un.org/dataportalapi/api/v1"
    target = base_url + relative_path
    # Calls the API
    response = requests.get(target)
    # Reformats response into a JSON object
    j = response.json()
    # The block below will deal with paginated results.
    # If results not paginated, this will be skipped.
    try:
        # If results are paginated, they are transformed into a
        # python dictionary.
        # The data may be accessed using the 'data' key of the dictionary.
        df = pd.json_normalize(j["data"])
        # As long as the nextPage key of the dictionary contains an
        # address for the next API call, the function will continue
        # to call the API and append the results to the dataframe.
        while j["nextPage"] is not None:
            response = requests.get(j["nextPage"])
            j = response.json()
            df_temp = pd.json_normalize(j["data"])
            df = pd.concat([df, df_temp])
    except Exception:
        if topic_list:
            df = pd.json_normalize(j, "indicators")
        else:
            df = pd.DataFrame(j)
    return df

In [9]:
df_locations = callAPI("/locations/")

In [16]:
df_locations.head()

,id,parentId,name,iso3,iso2,locationTypeId,locationType,longitude,latitude
0,4,5501,Afghanistan,AFG,AF,4,Country,67.709953,33.939110
1,8,925,Albania,ALB,AL,4,Country,20.168331,41.153332
2,12,912,Algeria,DZA,DZ,4,Country,1.659626,28.033886
3,16,957,American Samoa,ASM,AS,4,Country,-170.696182,-14.306021
4,20,925,Andorra,AND,AD,4,Country,1.521801,42.506287


In [17]:
# Identifies ID code for Western Africa
western_africa_id = df_locations.loc[df_locations["name"] == "Afghanistan", "id"].iloc[
    0
]

In [18]:
western_africa_id

4